In [ ]:
import face_recognition
import csv
import datetime
from pathlib import Path
import cv2
import dlib
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
import os

In [ ]:
pretrained_model = "https://github.com/yu4u/age-gender-estimation/releases/download/v0.5/weights.28-3.73.hdf5"
modhash = 'fbe63257a054c1c5466cfd7bf14646d6'

In [ ]:
result_file = open(os.getcwd()+"/"+"results.csv", "a")
myFields = ['Date', 'Time', 'FaceID', 'Gender', 'Age']

writer = csv.DictWriter(result_file, fieldnames=myFields)
writer.writeheader()

In [ ]:
depth = 16
k = 8
weight_file = None
margin = 0.4
image_dir = ""

In [ ]:
if not weight_file:
        weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                               file_hash=modhash, cache_dir=str(Path(__file__).resolve().parent))

In [ ]:
detector = dlib.get_frontal_face_detector()

In [ ]:
# load model and weights
img_size = 64
#img_size = 128
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)
image_path="/home/sensus/Development/age-gender-estimation/faces_image/"
faces_path="/media/sensus/DF76-0250/DriveDevelopment/Development/age-gender-estimation/faces/"

In [ ]:
index= 0
for path in Path(image_dir).glob("*.*"):
        #print(path)
        name = str(path).split("/")[-1]
        #print(str(name))
        img = cv2.imread(str(path),1)

        if img is not None:
            h,w, _ = img.shape
            r = 640 /max(w,h)
            '''
            face_landmarks_list = face_recognition.face_landmarks(img)
            print(str(face_landmarks_list))
            '''
            #cv2.resize(img,(int(w*r),int(h*r)))
            
            input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            img_h, img_w, _ = np.shape(input_img)

            # detect faces using dlib detector
            detected = detector(input_img, 1)
            #print(detected)
            faces = np.empty((len(detected), img_size, img_size, 3))
            if len(detected) > 0:
                for i, d in enumerate(detected):
                    x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
                    xw1 = max(int(x1 - margin * w), 0)
                    yw1 = max(int(y1 - margin * h), 0)
                    xw2 = min(int(x2 + margin * w), img_w - 1)
                    yw2 = min(int(y2 + margin * h), img_h - 1)
                    print('xw1 :'+str(xw1)+'yw1 :'+str(yw1)+'xw2 :'+str(xw2)+'yw2 :'+str(yw2))
                    faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
                    '''
                    print(str(name))
                    m = str(name).split('.')[0]
                    index = 0
                
                    for f in faces:
                        cv2.imwrite(os.path.join(faces_path,str(m)+"_face-"+str(index)+".jpg"),f)
                        #print(f)
                        index = index + 1
                    '''
                    print("___________________________________________")
                    fll = face_recognition.face_landmarks(img)
                    le = 0
                    if (len(fll) > 0):
                        x = fll[0]
                        #print(str(x))
                        for key in x:
                           le = le + len(x[key])
                    else:
                        pass

                    print(le)
                    if (le == 72):
                        # predict ages and genders of the detected faces
                        
                        results = model.predict(faces)
                        predicted_genders = results[0]
                        ages = np.arange(0, 101).reshape(101, 1)
                        predicted_ages = results[1].dot(ages).flatten()
                        
                        print(predicted_ages[0])
                        #index = 0
                        m = str(name).split('.')[0]
                        d = m.split('_')[1]
                        h = m.split('_')[2]
                        t = int(int(m.split('image')[1])/60)
                        s = int(int(m.split('image')[1])%60)
                        e = str(h)+":"+str(t)+":"+str(s)
                        print(str(name))
                        
                        '''
                        result_face = model.predict(f)
                        predicted_genders = result_face[0]
                        ages = np.arange(0,101).reshape(101,1)
                        predicted_ages_face = result_face[1].dot(ages).flatten()
                        '''
                        crop_img = img[yw1:yw2 + 1, xw1:xw2 + 1]
                        #cv2.imwrite()
                        ids = (str(m)+"_face-"+str(i)+"_"+str("M" if predicted_genders[index][0] < 0.5 else "F")+"_"+str(round(predicted_ages[i])))
                        print(ids)
                        cv2.imwrite(os.path.join(faces_path,str(m)+"_face-"+str(i)+"_"+str("M" if predicted_genders[index][0] < 0.5 else "F")+"_"+str(round(predicted_ages[i]))+".jpg"), crop_img)
                        print("___________________________________________")


                        print(predicted_ages[i])
                        print(predicted_genders)
                        print("M" if predicted_genders[i][0] < 0.5 else "F")
                        print("___________________________________________")

                        cv2.imwrite(os.path.join(image_path,str(name)),img)
                    else:
                        pass